In [5]:
from tqdm.autonotebook import tqdm, trange

In [2]:
## functional dependencies
import time

## setting up env
import os
from dotenv import load_dotenv
from numpy.core.defchararray import endswith
load_dotenv()

## LangChain dependencies
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_groq import ChatGroq
from langchain_chroma import Chroma
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_cohere.chat_models import ChatCohere
## LCEL implementation of LangChain ConversationalRetrievalChain
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\souga\AppData\Local\sagemaker\sagemaker\config.yaml


In [3]:
import os
from pathlib import Path

# For both .py and .ipynb files
try:
    # For .py files
    current_dir = os.path.dirname(os.path.abspath(__file__))
except NameError:
    # For .ipynb files
    current_dir = os.getcwd()

data_path = os.path.join(current_dir, "data")
persistent_directory = os.path.join(current_dir, "data-ingestion-local")


llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.15)

In [6]:
## open-source embedding model from HuggingFace - taking the default model only
embedF = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")

## loading the vector database from local
vectorDB = Chroma(embedding_function=embedF, persist_directory=persistent_directory)

## setting up the retriever
kb_retriever = vectorDB.as_retriever(search_type="similarity",search_kwargs={"k": 3})


In [7]:
contextualize_q_system_prompt = (
    """
        TASK: Convert context-dependent questions into standalone queries.

        INPUT: 
        - chat_history: Previous messages
        - question: Current user query

        RULES:
        1. Replace pronouns (it/they/this) with specific referents
        2. Expand contextual phrases ("the above", "previous")
        3. Return original if already standalone
        4. NEVER answer or explain - only reformulate

        OUTPUT: Single reformulated question, preserving original intent and style.

        Example:
        History: "Let's discuss Python."
        Question: "How do I use it?"
        Returns: "How do I use Python?"
    """
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm = llm,
    retriever = kb_retriever,
    prompt = contextualize_q_prompt
)

In [15]:
## setting-up the prompt
system_prompt_template = (
    "As a Legal Assistant Chatbot specializing in legal queries, "
    "your primary objective is to provide accurate and concise information based on user queries. "
    "You will adhere strictly to the instructions provided, offering relevant "
    "context from the knowledge base while avoiding unnecessary details. "
    "Your responses will be brief, to the point, concise and in compliance with the established format. "
    "If a question falls outside the given context, you will simply output that you are sorry and you don't know about this. "
    "The aim is to deliver professional, precise, and contextually relevant information pertaining to the context. "
    "Use four sentences maximum. "
    "\nCONTEXT: {context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_template),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

In [16]:
qa_prompt

ChatPromptTemplate(input_variables=['context', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMes

In [21]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


chat_history = []

question = "What are the conditions that must be fulfilled for a marriage to be solemnized in India?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
print("User: ", question)
print("AI: ", ai_msg_1["answer"])
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "Can you specify if there are any age requirements for the parties involved?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print("User: ", second_question)
print("AI: ", ai_msg_2["answer"])

User:  What are the conditions that must be fulfilled for a marriage to be solemnized in India?
AI:  To solemnize a marriage in India, the following conditions must be fulfilled: 

The parties and three witnesses must sign a declaration in the presence of the Marriage Officer, and the declaration must be countersigned by the Marriage Officer. 
The marriage must be solemnized at the office of the Marriage Officer or at a place within a reasonable distance, with conditions and additional fees as prescribed. 
The marriage must be in a form chosen by the parties, but it must be complete and binding only if each party says to the other, in the presence of the Marriage Officer and witnesses, that they take the other as their lawful wife or husband.
User:  Can you specify if there are any age requirements for the parties involved?
AI:  According to the Special Marriage Act, 1954, the parties involved in a marriage must be at least 21 years of age for males and 18 years of age for females.
